In [ ]:
from glob import glob
from PipeLine import *
from dask.distributed import Client
from dask import delayed
from dask import compute
client = Client(n_workers=4)
client

#### Bugs in  the python version of this file; this notebook work fine.
- This notebook cannot convert to a python file since I do not know how to efficently use dask in a *.py* file. I try to solve this issue by configuring ask apprportiately there. Check the file.
- Memeroy leak warning and error.

In [ ]:
# analyze all the bug groups at once:
database = '/Users/amirhsi_mini/' # parent path
simulation_type = 'bug' # bug or all
input_db_name = "extraction"
input_sim_groups = glob(database+input_db_name+"/N*-extraction/") # Path to bug extraction folders/files
output_db_name = "analysis"
geometry = 'cylindrical'
analysis_delayed = []
for input_sim_group in input_sim_groups:
    analysis = delayed(PipeLine.whole_group_analysis_bug)(input_sim_group, input_db_name, output_db_name, simulation_type, geometry)
    analysis_delayed.append(analysis)

In [ ]:
%%time
# it takes less than 5 min for 12 simulation groups with 4 workers.
results = compute(analysis_delayed)

# Properties all_in_one files

In [ ]:
# create one dataframe of all the properties files:
database_path = database+output_db_name+"/"
properties_files = glob(database_path+"N*-"+simulation_type+"-"+output_db_name+"/N*-properties.csv")
properties_files = PipeLine.file_reader(properties_files,extensions=['-properties.csv'])
properties_all_in_one = PipeLine.all_in_one_properties(properties_files, save_to=database_path, round_to=4, index_col=0)
properties_files_ens_avg = glob(database_path+"N*-"+simulation_type+"-"+output_db_name+"-ens_avg/N*-properties-ens_avg.csv")
properties_files_ens_avg = PipeLine.file_reader(properties_files_ens_avg,extensions=['-properties-ens_avg.csv'])
properties_all_in_one_ens_avg = PipeLine.all_in_one_properties(properties_files_ens_avg, ens_avg=True, norm_func=PipeLine.cylindrical_norm, save_to=database_path, round_to=4, index_col=0)

### ACF (This section should be used for older version of whole_group_analysis_bug in which acf routines were not added)

In [ ]:
def whole_group_acf_bug(extraction_db, extraction_db_name, analysis_db_name, simulation_type, geometry):
    sim_csvs = glob(extraction_db+"/N*/N*.csv")
    group_type = simulation_type+'-'+analysis_db_name
    analysis_save_to = PipeLine.group_new_path(extraction_db, extraction_db_name, analysis_db_name, group_type)
    analysis_ens_avg_save_to = PipeLine.group_new_path(extraction_db, extraction_db_name, analysis_db_name, group_type, ens_avg=True)
    analysis_sumrule_bug_acf(sim_csvs, geometry, analysis_save_to, analysis_ens_avg_save_to)

In [ ]:
def analysis_sumrule_bug_acf(sim_csvs, geometry, analysis_save_to, analysis_ens_avg_save_to):
    nlags = 7000
    alpha = 0.05
    fsd_name = 'fsd_t'
    fsd_files = PipeLine.file_reader(sim_csvs, extensions=['-fsd_t.csv'])
    fsd_dicts = PipeLine.ensemble(fsd_files, fsd_name, geometry, save_to=None)
    fsd_dict_ens_avg = PipeLine.group(fsd_dicts,fsd_name,geometry,save_to=None)
    acf_dicts, acf_lower_dicts, acf_upper_dicts = PipeLine.acf_generator(fsd_dicts, fsd_name, geometry, nlags=nlags, alpha=alpha,  save_to=analysis_save_to)
    _ = PipeLine.group(acf_dicts, fsd_name+'-acf_only', geometry, save_to=analysis_ens_avg_save_to)
    _ = PipeLine.group(acf_lower_dicts, fsd_name+'-acf_ci_upper', geometry, save_to=analysis_ens_avg_save_to)
    _ = PipeLine.group(acf_upper_dicts, fsd_name+'-acf_ci_lower', geometry, save_to=analysis_ens_avg_save_to)
    
    gyr_name = 'gyr_t'
    gyr_files = PipeLine.file_reader(sim_csvs,extensions=['-gyr_t.csv'])
    gyr_dicts = PipeLine.ensemble(gyr_files,gyr_name, geometry, save_to=None)
    _ = PipeLine.group(gyr_dicts, gyr_name, geometry, save_to=None)
    acf_dicts, acf_lower_dicts, acf_upper_dicts = PipeLine.acf_generator(gyr_dicts, gyr_name, geometry, nlags=nlags, alpha=alpha,  save_to=analysis_save_to)
    _ = PipeLine.group(acf_dicts, gyr_name+'-acf_only', geometry, save_to=analysis_ens_avg_save_to)
    _ = PipeLine.group(acf_lower_dicts, gyr_name+'-acf_ci_upper', geometry, save_to=analysis_ens_avg_save_to)
    _ = PipeLine.group(acf_upper_dicts, gyr_name+'-acf_ci_lower', geometry, save_to=analysis_ens_avg_save_to)
    
    rFlory_name = 'rFlory_t'
    rFlory_files = PipeLine.file_reader(sim_csvs,extensions=['-rFlory_t.csv'])
    rFlory_dicts = PipeLine.ensemble(rFlory_files,rFlory_name,geometry,save_to=None)
    _ = PipeLine.group(rFlory_dicts,rFlory_name,geometry,save_to=None)
    acf_dicts, acf_lower_dicts, acf_upper_dicts = PipeLine.acf_generator(rFlory_dicts, rFlory_name, geometry, nlags=nlags, alpha=alpha,  save_to=analysis_save_to)
    _ = PipeLine.group(acf_dicts, rFlory_name+'-acf_only', geometry, save_to=analysis_ens_avg_save_to)
    _ = PipeLine.group(acf_lower_dicts, rFlory_name+'-acf_ci_upper', geometry, save_to=analysis_ens_avg_save_to)
    _ = PipeLine.group(acf_upper_dicts, rFlory_name+'-acf_ci_lower', geometry, save_to=analysis_ens_avg_save_to)

In [ ]:
database = '/Users/amirhsi_mini/' # parent path
simulation_type = 'bug' # bug or all
input_db_name = "extraction"
input_sim_groups = glob(database+input_db_name+"/N*-extraction/") # Path to bug extraction folders/files
output_db_name = "analysis"
geometry = 'cylindrical'
analysis_delayed = []
for input_sim_group in input_sim_groups:
    analysis = delayed(whole_group_acf_bug)(input_sim_group, input_db_name, output_db_name, simulation_type, geometry)
    analysis_delayed.append(analysis)

In [ ]:
%%time
# it takes less than 5 min for 12 simulation groups with 4 workers.
results = compute(analysis_delayed)